# RSI with Stop loss trigger on 1.5%

In [ ]:
from binance.client import Client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime
import time

api_key = 'Your Api_Key'
api_secret = 'Your Api_secret'

client = Client(api_key, api_secret)

def trade(symbol):
    def getminutedatal(symbol):
        frame = pd.DataFrame(client.get_historical_klines(symbol, '3m', '600 min ago UTC'))
        frame = frame[[0, 4]]
        frame.columns = ["Time", "Close"]
        frame.set_index("Time", inplace=True)
        frame.index = pd.to_datetime(frame.index, unit='ms')
        frame.Close = frame.Close.astype(float)
        return frame
    
    def rsi_tradingview(ohlc: pd.DataFrame, period: int = 6, round_rsi: bool = True):
        delta = ohlc["Close"].diff()
        up = delta.copy()
        up[up < 0] = 0
        up = pd.Series.ewm(up, alpha=1 / period).mean()
        down = delta.copy()
        down[down > 0] = 0
        down *= -1
        down = pd.Series.ewm(down, alpha=1 / period).mean()
        rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))
        return np.round(rsi, 2) if round_rsi else rsi

    def applyindicator(df):
        df['SMA200'] = df.Close.rolling(200).mean()
        df['SMA20'] = df.Close.rolling(20).mean()
        df['Stddev'] = df.Close.rolling(20).std()
        df['Upper'] = df.SMA20 + 2 * df.Stddev
        df['Lower'] = df.SMA20 - 2 * df.Stddev
        df['RSi'] = rsi_tradingview(df)
    
    def implement_rsi_strategy(prices, rsi):
        buy_price = []
        sell_price = []
        rsi_signal = []
        signal = 0

        for i in range(len(rsi)):
            if rsi[i] < 30:
                if signal != 1:
                    buy_price.append(prices[i])
                    sell_price.append(np.nan)
                    signal = 1
                    rsi_signal.append(signal)
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    rsi_signal.append(0)
            elif rsi[i] > 70:
                if signal != -1:
                    buy_price.append(np.nan)
                    sell_price.append(prices[i])
                    signal = -1
                    rsi_signal.append(signal)
                else:
                    buy_price.append(np.nan)
                    sell_price.append(np.nan)
                    rsi_signal.append(0)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                rsi_signal.append(0)

        return buy_price, sell_price, rsi_signal

    def price_calc(symbol, limit=0.985):
        raw_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
        return raw_price * limit
    
    def rightrounding(lotsize):
        splitted = str(lotsize).split(".")
        if float(splitted[0]) == 1:
            return 0
        else:
            return len(splitted[1])

    def quantity_calc(symbol, investment, step_size):
        price = price_calc(symbol)
        qty = round(investment / price, int(-np.log10(step_size)))
        return qty

    def round_price(price, tick_size):
        return round(price / tick_size) * tick_size

    # Fetch exchange info to get filters and precision
    exchange_info = client.get_exchange_info()
    symbol_info = next(item for item in exchange_info['symbols'] if item['symbol'] == symbol)
    price_filter = next(filter(lambda f: f['filterType'] == 'PRICE_FILTER', symbol_info['filters']))
    min_price = float(price_filter['minPrice'])
    max_price = float(price_filter['maxPrice'])
    tick_size = float(price_filter['tickSize'])
    lot_size_filter = next(filter(lambda f: f['filterType'] == 'LOT_SIZE', symbol_info['filters']))
    min_qty = float(lot_size_filter['minQty'])
    step_size = float(lot_size_filter['stepSize'])

    # Main trading logic
    while True:
        df = getminutedatal(symbol)
        applyindicator(df)
        buy_price, sell_price, rsi_signal = implement_rsi_strategy(df['Close'], df['RSi'])
        df["buy_price"] = buy_price
        df["Sell_price"] = sell_price
        df["Rsi_signal"] = rsi_signal  
        df['buy'] = df['Rsi_signal'].apply(lambda x: 1 if x == 1 else 0)
        df['Sell'] = df['Rsi_signal'].apply(lambda x: 1 if x == -1 else 0)
        pd.set_option('display.max_rows', 5)
        pd.set_option('display.max_columns', 34)
        display(df)
        today = date.today()
        df.to_csv(f"Dataframe{today}.csv", index=False)
        
        for x, y in df[["buy", "Sell"]].take([-2]).values:
            try:
                if x == 1:
                    # Place a buy order
                    buy_order = client.create_order(
                        symbol=symbol,
                        side="BUY",
                        type="MARKET",
                        quantity=quantity_calc(symbol, 85, step_size)
                    )
                    print(buy_order)
                    executed_price = float(buy_order['fills'][0]['price'])
                    stop_sell_order_price = executed_price * 0.985

                    # Ensure stop_sell_order_price is within the allowed range and round it
                    stop_sell_order_price = max(min(stop_sell_order_price, max_price), min_price)
                    stop_sell_order_price = round_price(stop_sell_order_price, tick_size)
                    
                    print(f"Stop Sell Order Price: {stop_sell_order_price}")
                    stop_limit_sell_order = client.create_order(
                        symbol=symbol,
                        side="SELL",
                        type="STOP_LOSS_LIMIT",
                        timeInForce="GTC",
                        stopPrice=stop_sell_order_price,
                        price=stop_sell_order_price,
                        quantity=quantity_calc(symbol, 85, step_size)
                    )
                    print(stop_limit_sell_order)
                            
                elif y == 1:
                    orders = client.get_open_orders(symbol=symbol) 
                    for order in orders:                    
                        ordId = order["orderId"]
                        client.cancel_order(symbol=symbol, orderId=ordId)
                    # Place a sell order
                    sell_order = client.create_order(
                        symbol=symbol, 
                        side="SELL", 
                        type="MARKET", 
                        quantity=quantity_calc(symbol, 85, step_size)
                    )
                    print(sell_order)
                else:
                    print("Do Nothing.....")
            except Exception as e:
                if "insufficient balance" in str(e).lower():
                    print("Insufficient balance error occurred. Logging and continuing...")
                elif "Timestamp" in str(e):
                    print("Timestamp error occurred. Synchronizing time...")
                    sync_time()
                else:
                    print(f"An error occurred: {e}")
        
        # Plotting the results
        ax1 = plt.subplot2grid((10, 1), (0, 0), rowspan=4, colspan=1)
        ax2 = plt.subplot2grid((10, 1), (5, 0), rowspan=4, colspan=1)
        ax1.plot(df['Close'], linewidth=2.5, color='skyblue', label='DF')
        ax1.plot(df.index, buy_price, marker='^', markersize=10, color='green', label='BUY SIGNAL')
        ax1.plot(df.index, sell_price, marker='v', markersize=10, color='red', label='SELL SIGNAL')
        ax1.set_title('Trading View RSI TRADE SIGNALS')
        ax2.plot(df['RSi'], color='orange', linewidth=2.5)
        ax2.axhline(30, linestyle='--', linewidth=1.5, color='grey')
        ax2.axhline(70, linestyle='--', linewidth=1.5, color='grey')
        plt.show()

        # Sleep before the next iteration to avoid making too many requests
        time.sleep(180)  # Sleep for 3 minutes



In [ ]:
trade('ETHUSDT') 